In [1]:
import cv2
import numpy as np

### Cutting Video and Produce templates and then i crop the pictures

In [2]:
#vid = cv2.VideoCapture('car.mp4') 

#frame_number = 0

#while True:
#    ret, frame = vid.read()
    
#    if not ret:
#        break
        
#    if frame_number % 10 == 0:
#        cv2.imwrite(f'templates/base/frame_{frame_number + 1}.png', frame)
    
#    cv2.imshow('frame', frame)
#    frame_number = frame_number + 1

#    if cv2.waitKey(1)==27:
#        break
    
#vid.release()
#cv2.destroyAllWindows()

#print(frame_number)

## Main Part of object tracjing

In [3]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [4]:
def resize_img(img, percentage):
    width = int(img.shape[1] *  percentage / 100)
    height = int(img.shape[0] *  percentage / 100)
    dim = (width, height)
    
    resized_img = cv2.resize(img, dim)
    return resized_img

In [5]:
#Frame Numbers that you select for templates!

template_frames = [1, 11, 21, 31, 41, 51, 51, 81 , 81, 91, 101, 111, 121, 131, 141, 151, 161, 161, 181, 191, 201, 211, 221, 231, 231,231,231, 271, 281, 291, 301, 301, 321, 331, 341,361, 361,371, 381, 391, 401, 411, 421]

In [6]:
templates = []
keypoints_templates = []
descriptors_templates = []

for template_number in template_frames:
    f = 'frame_{}.png'.format(template_number)
    #print(f)
    
    template_ = cv2.imread('templates/{}'.format(f)) # , 0)
    template_ = resize_img(template_ , 50)
    keypoints_template_ , descriptors_template_ = sift.detectAndCompute(template_, None)
    
    templates.append(template_)
    keypoints_templates.append(keypoints_template_)    
    descriptors_templates.append(descriptors_template_)    

#print(templates)

In [7]:
cap = cv2.VideoCapture('car.mp4') 

In [8]:
frame_number = 0

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = resize_img(frame, 50)
    keypoints_frame, descriptors_frame = sift.detectAndCompute(frame, None) 
    
    frame_number = frame_number + 1
    
    template_number = frame_number // 10
    #print(template_number)
    
    template = templates[template_number]
    keypoints_template = keypoints_templates[template_number]
    descriptors_template = descriptors_templates[template_number]
    
    matches = bf.knnMatch(descriptors_template, descriptors_frame, k=2)
    
    good_matches = []
    
    if matches is not None:
        
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)

        if len(good_matches) >= 4:

            src_pts = np.float32([keypoints_template[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2) 
            dst_pts = np.float32([keypoints_frame[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            
            #print('src_pts ' , src_pts)
            #print('dst_pts' , dst_pts)

            M,_=cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)
            
            h, w, _ = template.shape
         
            if M is not None:
                template_corner = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)

                transformed_corner = cv2.perspectiveTransform(template_corner, M)

                bounding_box = cv2.polylines(frame, [np.int32(transformed_corner)], True, (0, 255, 0), 2)
                cv2.imshow('bounding box', bounding_box)
    
    if cv2.waitKey(1) == 27:
        break
        
cap.release()
cv2.destroyAllWindows()